In [1]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import *

spark = SparkSession \
    .builder \
    .master("spark://spark-master:7077") \
    .appName("PemiSpark") \
    .config("spark.sql.warehouse.dir", "/tmp/data/spark-warehouse") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()
    

sc = spark.sparkContext
sc.setLogLevel("INFO")

# do something to prove it works
rdd = sc.parallelize(range(100000000))
rdd.sumApprox(3)



4999999950000000.0

In [2]:
# Read some local data
sc.textFile("/tmp/data/Kiosk-OCC_201709140800.csv").count()

139

In [3]:
# Convert a pandas dataframe to a spark dataframe and then back again

pd_df = pd.DataFrame({
    'id': [1,2,3],
    'name': ['one', 'two', 'three']
})

spark_df = spark.createDataFrame(pd_df)
spark_df.show()

spark_df.toPandas()

+---+-----+
| id| name|
+---+-----+
|  1|  one|
|  2|  two|
|  3|three|
+---+-----+



,id,name
0,1,one
1,2,two
2,3,three


In [4]:
# Build some dummy data via Pemi and do operations in spark

import pemi
import pemi.data

    

sales_schema = {
    'beer_id':  {'ftype': 'integer', 'required': True},
    'sold_at':  {'ftype': 'date', 'in_format': '%m/%d/%Y', 'required': True},
    'quantity': {'ftype': 'integer', 'required': True}
}

beers_schema = {
    'id':       {'ftype': 'integer', 'required': True},
    'name':     {'ftype': 'string', 'required': True},
    'style':    {'ftype': 'string'},
    'abv':      {'ftype': 'float'},
    'price':    {'ftype': 'decimal', 'precision': 16, 'scale': 2}
}
 
sales_table = pemi.data.Table(
    '''
    | beer_id | sold_at    | quantity |
    | -       | -          | -        |
    | 1       | 01/01/2017 | 3        |
    | 2       | 01/02/2017 | 3        |
    | 3       | 01/03/2017 | 5        |
    | 4       | 01/04/2017 | 8        |
    | 5       | 01/04/2017 | 6        |
    | 1       | 01/06/2017 | 1        |
    ''',
    schema=sales_schema,
    fake_with={
        'beer_id': { 'valid': lambda: pemi.data.fake.random_int(1,4) },
        'sold_at': { 'valid': lambda: pemi.data.fake.date_time_this_decade().date() },
        'quantity': {'valid': lambda: pemi.data.fake.random_int(1,100) },
        'bumpkin': { 'valid': lambda: pemi.data.fake.word(['bumpkin A', 'bumpkin B', 'bumpkin C']) }
    }
)


beers_table = pemi.data.Table(
    '''
    | id | name          | style |
    | -  | -             | -     |
    | 1  | SpinCyle      | IPA   |
    | 2  | OldStyle      | Pale  |
    | 3  | Pipewrench    | IPA   |
    | 4  | AbstRedRibbon | Lager |
    ''',
    schema=beers_schema,
    fake_with={
        'abv': {'valid': lambda: pemi.data.fake.pydecimal(2, 2, positive=True)},
        'price': {'valid': lambda: pemi.data.fake.pydecimal(2, 2, positive=True)}
    }
)

beers_df = spark.createDataFrame(beers_table.df)
sales_df = spark.createDataFrame(sales_table.df)
sales_df.show()
beers_df.show()


beers_df.createOrReplaceTempView('beers')
sales_df.createOrReplaceTempView('sales')

beer_sales_df = spark.sql('''
    SELECT
      sales.beer_id,
      beers.name,
      beers.style,
      sales.sold_at,
      sales.quantity,
      CAST(beers.price as decimal(10, 2)) AS unit_price,
      CAST(beers.price * sales.quantity AS DECIMAL(10,2)) AS sell_price
    FROM
      sales
    LEFT JOIN
      beers
    ON
      sales.beer_id = beers.id
''')

beer_sales_df.show()

+-------+----------+--------+
|beer_id|   sold_at|quantity|
+-------+----------+--------+
|      1|2017-01-01|       3|
|      2|2017-01-02|       3|
|      3|2017-01-03|       5|
|      4|2017-01-04|       8|
|      5|2017-01-04|       6|
|      1|2017-01-06|       1|
+-------+----------+--------+

+---+-------------+-----+--------------------+--------------------+
| id|         name|style|                 abv|               price|
+---+-------------+-----+--------------------+--------------------+
|  1|     SpinCyle|  IPA|52.72000000000000...|25.90000000000000...|
|  2|     OldStyle| Pale|33.43000000000000...|6.970000000000000000|
|  3|   Pipewrench|  IPA|80.97000000000000...|94.00000000000000...|
|  4|AbstRedRibbon|Lager|96.97000000000000...|74.95000000000000...|
+---+-------------+-----+--------------------+--------------------+

+-------+-------------+-----+----------+--------+----------+----------+
|beer_id|         name|style|   sold_at|quantity|unit_price|sell_price|
+-------+-

In [14]:
%%time
# Read some large data from CSV files (10M records) that I've confirmed is too large for Pandas

headers = [
    'rad_key',
    'distributor_key',
    'retailer_key',
    'item_key',
    'txn_date',
    'invoice_nbr',
    'outlet_key',
    'distributor_sales_rep_key',
    'distributor_item_nbr',
    'quantity_cases',
    'quantity_bottles',
    'bottles_per_case',
    'physical_cases',
    'nine_liters_per_physical_case',
    'nine_liter_cases',
    'ext_price',
    'ext_price_2',
    'bottle_deposit_amt',
    'tax_amt',
    'additional_charges_amt',
    'source_file',
    'row_hash',
    'rpt_current_ind',
    'rpt_from_dt',
    'rpt_thru_dt'    
]

fields = [StructField(field_name, StringType(), True) for field_name in headers]
schema = StructType(fields)

df = spark.read.csv("/tmp/data/largeRAD", header=False, sep='|', schema=schema)
df.createOrReplaceTempView('rad')

#df.write.csv('/tmp/data/largeRADOut.csv', mode='overwrite', header=False)

pd_df = spark.sql('SELECT distributor_key, COUNT(*) AS n, SUM(ext_price) AS ext_price FROM rad GROUP BY 1').toPandas()


CPU times: user 10 ms, sys: 10 ms, total: 20 ms
Wall time: 13.8 s
